In [2]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types
import os
import asyncio

In [3]:
try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Authentication & API setup completed.")
except Exception as e:
  print(f"Errror: {e}")

Authentication & API setup completed.


In [4]:
# retry_options

retry_config = types.HttpRetryOptions(
  attempts=5,
  initial_delay=1,
  exp_base=7,
  http_status_codes=[429,500,503,504]
)

### Helper function

Helper function that manages a complete conversation session, handling session
creation/retrieval, query processing, and response streaming. It supports
both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```

In [5]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name : str ="default"

):
  print(f"\n ### SESSION: {session_name}")

  #Get APP name from the Runner
  app_name = runner_instance.app_name

  #Attempt to create a new session or retrieve an existing one
  try:
    session = await session_service.create_session(
      app_name=app_name, user_id=USER_ID, session_id=session_name
    )
  except:
    session= await session_service.get_session(
      app_name=app_name,user_id =USER_ID,session_id=session_name
    )

  #Process queries if provided
  if user_queries:
    #convert single query to list for uniform processing
    if type(user_queries)==str:
      user_queries=[user_queries]

    #Process each query in the list sequentially
    for query in user_queries:
      print(f"\n User > {query}")

      #Convert the query string to the ADK content format
      query=types.Content(role="user",parts=[types.Part(text=query)])

      #stream the agents response asynchronously
      async for event in runner_instance.run_async(
        user_id=USER_ID, session_id=session.id, new_message=query
      ):
        #check if the event contains valid content
        if event.content and event.content.parts:
          #filter out empty of None responses before printing
          if(
            event.content.parts[0].text !="None"
            and event.content.parts[0].text
          ):
            print(f"{MODEL_NAME} > ", event.content.parts[0].text)
  else:
    print("No queries!")

In [6]:
# Implementing Stateful Agent
#he can remember and have constructive conversations.

APP_NAME = "default" #Application
USER_ID = "default" #User
SESSION = "default" #Session
MODEL_NAME = "gemini-2.5-flash-lite"

#step1 :create llm agent

root_agent = Agent(
  model=Gemini(model=MODEL_NAME,retry_options=retry_config),
  name="text_chat_bot",
  description="A text chatbot", #description of the agents purpose
)

#Step2: set up session management
#InMemorySessionService stores conversations in RAM temporary.

session_service = InMemorySessionService()

#Step3: create the runner
runner=Runner(agent=root_agent,app_name=APP_NAME,session_service=session_service)

print("Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


In [6]:
### Testing the Stateful agent
await run_session(
  runner,
  [
    "Hi,Im Ilias, whats the climate change Risk?",
    "Whats my name?",#this time , the agent should remember
  ],
  "stateful-agentic-session"
)


 ### SESSION: stateful-agentic-session

 User > Hi,Im Ilias, whats the climate change Risk?
gemini-2.5-flash-lite >  Hi Ilias! It's great to meet you.

Climate change risk refers to the potential for negative impacts on human societies and natural systems as a result of changes in the Earth's climate. It's essentially the likelihood of experiencing harmful consequences due to global warming and its related effects.

Here's a breakdown of what that means:

*   **Climate Change:** This is the long-term shift in global or regional climate patterns, primarily driven by increased levels of atmospheric greenhouse gases produced by the use of fossil fuels.
*   **Risk:** This involves two main components:
    *   **Hazard:** The physical event itself (e.g., extreme heat, sea-level rise, more intense storms).
    *   **Vulnerability and Exposure:** How susceptible people, ecosystems, and infrastructure are to these hazards, and whether they are located in harm's way.

**So, climate change risk

#### Restarting the kernel, the agent wont remember anything

In [7]:
await run_session(
    runner,
    ["What did I ask you about earlier?", "And remind me, what's my name?"],
    "stateful-agentic-session",
)  # Note, we are using same session name


 ### SESSION: stateful-agentic-session

 User > What did I ask you about earlier?
gemini-2.5-flash-lite >  I am sorry, but I cannot recall past conversations. My memory is limited to the current interaction.

 User > And remind me, what's my name?
gemini-2.5-flash-lite >  I do not have access to your personal information, including your name. I am a text chatbot and do not store any user data.
